In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as io
import pandas as pd
from pandas import DataFrame as dframe
import cvxpy as cv

In [3]:
def Dic(feat_l_tr,feat_r_tr): #入力の配列は2次元配列である必要がある
    feat_tr = np.concatenate([feat_l_tr, feat_r_tr],0)
    return feat_tr

In [4]:
#各クラスの特徴ベクトルの配列の列数を格納した配列を引数にすればもう少し拡張性が上がる
def SGRM(feat_tr,feat_tr_dic_1,feat_tr_dic_2,feat_tr_dic,feat_l_te,matrix_index,λ_1,λ_2):
    #λ_1 = 0.01
    #λ_2 = 0.01
    answer = np.zeros(feat_l_te.shape[1])
    #スパース性を格納する行列
    SGRM_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    #leftクラスのみのスパース性を格納するための行列
    SGRM_l_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    #rightクラスのみのスパース性を格納するための行列
    SGRM_r_Cla = np.zeros((feat_tr_dic.shape[1], feat_l_te.shape[1]))
    for i in range(feat_l_te.shape[1]):
        u = cv.Variable((feat_tr_dic.shape[1],1)) #目的関数の次元
        #SGRMの最適化問題を設定している
        objective = cv.Minimize(0.5*(cv.norm(feat_tr_dic@u-feat_l_te[:,i,np.newaxis],2)**2)
                                +λ_1*cv.norm(u[:,0],1)
                                +λ_2*(cv.norm(u[0:feat_tr.shape[0],0],2)
                                      +cv.norm(u[feat_tr.shape[0]:feat_tr_dic_1.shape[0],0],2)
                                     +cv.norm(u[feat_tr_dic_1.shape[0]:feat_tr_dic_2.shape[0],0],2)
                                     +cv.norm(u[feat_tr_dic_2.shape[0]:feat_tr_dic.shape[0],0],2)))
                                     #+cv.norm(u[feat_tr_dic_3.shape[0]:feat_tr_dic.shape[0],0],2)))
                                     #+cv.norm(u[feat_tr_dic_4.shape[0]:feat_tr_dic_5.shape[0],0],2)
                                     #+cv.norm(u[feat_tr_dic_5.shape[0]:feat_tr_dic_6.shape[0],0],2)
                                     #+cv.norm(u[feat_tr_dic_6.shape[0]:feat_tr_dic.shape[0],0],2)))
        #constraints = [u == 0.5*(cv.norm(feat_tr_dic@u-feat_l_te[:,i,np.newaxis],2)**2)+λ_1*cv.norm(u[:,0],1)
                       #+λ_2*(cv.norm(u[0:feat_tr.shape[0],0],2)
                             #+cv.norm(u[feat_tr.shape[0]:feat_tr_dic_1.shape[0],0],2)
                             #+cv.norm(u[feat_tr_dic_1.shape[0]:feat_tr_dic_2.shape[0],0],2)
                             #+cv.norm(u[feat_tr_dic_2.shape[0]:feat_tr_dic_3.shape[0],0],2)
                             #+cv.norm(u[feat_tr_dic_3.shape[0]:feat_tr_dic.shape[0],0],2))] 
                    
        prob = cv.Problem(objective)
        result = prob.solve()
        #classVariableから値を取るためには.valueを付ける
        SGRM_Cla[:,i] = np.squeeze(u.value)
        #各クラスに関連する表現ベクトルの値を抜き出す
        #lは仕切りの値
        l = 0
        j = 0
        while j < matrix_index.shape[0]-1:
            SGRM_l_Cla[l:l+matrix_index[j],i] = SGRM_Cla[l:l+matrix_index[j],i]     
            l = l + matrix_index[j]
            j = j + 1
            SGRM_r_Cla[l:l+matrix_index[j],i] = SGRM_Cla[l:l+matrix_index[j],i]
            l = l + matrix_index[j]
            j = j + 1
    #パラメータを使って擬似的に各クラスのテスト信号の特徴ベクトルを作成
    dout_feat_l_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))
    dout_feat_r_te = np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) 
    for i in range(feat_l_te.shape[1]):
        dout_feat_l_te[:,i] = feat_tr_dic@SGRM_l_Cla[:,i]
    for i in range(feat_l_te.shape[1]):
        dout_feat_r_te[:,i] = feat_tr_dic@SGRM_r_Cla[:,i]
    #作成した擬似特徴ベクトルとの残差を求める
    R_l =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1])) #残差を格納するための配列
    R_r =  np.zeros((feat_l_te.shape[0], feat_l_te.shape[1]))  
    for i in range(feat_l_te.shape[1]):
        R_l[:,i] = feat_l_te[:,i]-dout_feat_l_te[:,i]
        R_r[:,i] = feat_l_te[:,i]-dout_feat_r_te[:,i]
        #残差を比べてクラスを割り当てる
        if np.linalg.norm(R_l[:,i], ord=2) < np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 1
        elif np.linalg.norm(R_l[:,i], ord=2) > np.linalg.norm(R_r[:,i], ord=2):
            answer[i] = 2
    return answer

In [5]:
def SGRM_test(filename1,filename2,filename3,filename4,C1_K1,C2_K1,C1_K2,C2_K2,C1_K3,C2_K3):
    #対象被験者の特徴ベクトル(トレーニングデータ＆テストデータ)をMATLAB(自身で作成)から入手    feat_C1 = np.loadtxt(filename1,delimiter=",")
    feat_C2 = np.loadtxt(filename2,delimiter=",")
    feat_Test_C1 = np.loadtxt(filename3,delimiter=",")
    feat_Test_C2 = np.loadtxt(filename4,delimiter=",")
    #非対象被験者の特徴ベクトルをMATLABから入手
    feat_C1_K1 = np.loadtxt(C1_K1,delimiter=",")
    feat_C2_K1 = np.loadtxt(C2_K1,delimiter=",")
    feat_C1_K2 = np.loadtxt(C1_K2,delimiter=",")
    feat_C2_K2 = np.loadtxt(C1_K2,delimiter=",")
    feat_C1_K3 = np.loadtxt(C1_K3,delimiter=",")
    feat_C2_K3 = np.loadtxt(C1_K3,delimiter=",")

    #対象被験者の各クラスのトレーニングデータから取得した特徴ベクトルを結合
    feat_tr = Dic(feat_C1,feat_C2)
    #非対象被験者の特徴ベクトルも同様の処理を行う
    feat_tr_K1 = Dic(feat_C1_K1,feat_C2_K1)
    feat_tr_K2 = Dic(feat_C1_K2,feat_C2_K2)
    feat_tr_K3 = Dic(feat_C1_K3,feat_C2_K3)

    #それぞれの特徴ベクトルをつなげる
    #この時、2M×Nの行列にするために転置させる(2Mは特徴ベクトルの次元、Nはスパース表現を推定するために使用した特徴ベクトルの総数)
    feat_tr_dic_1 = Dic(feat_tr,feat_tr_K1)
    feat_tr_dic_2 = Dic(feat_tr_dic_1,feat_tr_K2)
    feat_tr_dic = Dic(feat_tr_dic_2,feat_tr_K3).T
    #feat_tr_dicはすべての特徴ベクトルをつなげたもの
    #テストデータの特徴ベクトルも2Mが行になるように転置させる
    feat_l_te = feat_Test_C1;
    feat_r_te = feat_Test_C2;
    feat_l_te = feat_l_te.T
    feat_r_te = feat_r_te.T
　　#各被験者の各クラスの特徴ベクトルの数を格納する(各クラスのみのスパース性を格納するための行列作成に使用)
    matrix_index = np.array([feat_C1.shape[0],feat_C2.shape[0],feat_C1_K1.shape[0],feat_C2_K1.shape[0],feat_C1_K2.shape[0],feat_C2_K2.shape[0],
                        feat_C1_K3.shape[0],feat_C2_K3.shape[0]])
    Max = 0
    #各ハイパーパラメーターでの正答率を格納
    weight = np.array([])
    #ハイパーパラメーターの値を0.01~0.1で0.01刻みで変更
    for i in range(1,11):
        λ_1 = i*0.01
        print(i)
        for j in range(1,11):
            λ_2 = j*0.01
            answer1 = SGRM(feat_tr,feat_tr_dic_1,feat_tr_dic_2,feat_tr_dic,feat_l_te,matrix_index,λ_1,λ_2)
            answer2 = SGRM(feat_tr,feat_tr_dic_1,feat_tr_dic_2,feat_tr_dic,feat_r_te,matrix_index,λ_1,λ_2)
            x1 = 0
            x2 = 0
            for i in range(answer1.shape[0]):
                if answer1[i] == 1:
                    x1 = x1 + 1
                if answer2[i] == 2:
                    x2 = x2 + 1
            #全体での正答率と各クラスでの正答率を求める
            temp = ((x1+x2)/(answer1.shape[0]+answer2.shape[0]))*100
            temp1 = ((x1)/(answer1.shape[0]))*100
            temp2 = ((x2)/(answer2.shape[0]))*100
            weight_l = np.array([temp,λ_1,λ_2])
            weight = np.append(weight,weight_l,axis=0)
            if temp > Max:
                Max = temp
                
    weight = weight.reshape([100, 3])                 
    return Max,temp1,temp2,weight

In [6]:
#train=20,test=40で分類を行った結果
probability = np.array([])
temp,temp1,temp2,weight_1 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A01T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A01T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A01T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A01T_20.txt"
                           ,"feat_2a_250Hz_30/feat_left_A02T_30.txt","feat_2a_250Hz_30/feat_right_A02T_30.txt","feat_2a_250Hz_30/feat_left_A03T_30.txt","feat_2a_250Hz_30/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30/feat_left_A09T_30.txt","feat_2a_250Hz_30/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_2 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A02T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A02T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A02T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A02T_20.txt"
                           ,"feat_2a_250Hz_30/feat_left_A01T_30.txt","feat_2a_250Hz_30/feat_right_A01T_30.txt","feat_2a_250Hz_30/feat_left_A03T_30.txt","feat_2a_250Hz_30/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30/feat_left_A09T_30.txt","feat_2a_250Hz_30/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_3 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A03T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A03T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A03T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A03T_20.txt"
                           ,"feat_2a_250Hz_30/feat_left_A02T_30.txt","feat_2a_250Hz_30/feat_right_A02T_30.txt","feat_2a_250Hz_30/feat_left_A01T_30.txt","feat_2a_250Hz_30/feat_right_A01T_30.txt"
                           ,"feat_2a_250Hz_30/feat_left_A09T_30.txt","feat_2a_250Hz_30/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_4 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A07T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A07T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A07T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A07T_20.txt"
                           ,"feat_2a_250Hz_30/feat_left_A03T_30.txt","feat_2a_250Hz_30/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30/feat_left_A01T_30.txt","feat_2a_250Hz_30/feat_right_A01T_30.txt","feat_2a_250Hz_30/feat_left_A09T_30.txt","feat_2a_250Hz_30/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_5 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A08T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A08T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A08T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A08T_20.txt"
                           ,"feat_2a_250Hz_30/feat_left_A03T_30.txt","feat_2a_250Hz_30/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30/feat_left_A01T_30.txt","feat_2a_250Hz_30/feat_right_A01T_30.txt","feat_2a_250Hz_30/feat_left_A09T_30.txt","feat_2a_250Hz_30/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)

probability_1 = probability.reshape([5, 3])
weight_1t = (weight_1[:,0]+weight_2[:,0]+weight_3[:,0]+weight_4[:,0]+weight_5[:,0])/5
print(probability_1)

1


/Users/kota/opt/anaconda3/lib/python3.9/site-packages/cvxpy/problems/problem.py:1337: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
[[ 51.25   0.   100.  ]
 [ 61.25  92.5   15.  ]
 [ 65.   100.    20.  ]
 [ 55.   100.     0.  ]
 [ 67.5   82.5   10.  ]]


In [134]:
#train=20,test=40で分類を行った結果
probability = np.array([])
temp,temp1,temp2,weight_1 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A01T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A01T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A01T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A01T_20.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A02T_30.txt","feat_2a_250Hz_30_2/feat_right_A02T_30.txt","feat_2a_250Hz_30_2/feat_left_A03T_30.txt","feat_2a_250Hz_30_2/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A09T_30.txt","feat_2a_250Hz_30_2/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_2 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A02T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A02T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A02T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A02T_20.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A01T_30.txt","feat_2a_250Hz_30_2/feat_right_A01T_30.txt","feat_2a_250Hz_30_2/feat_left_A03T_30.txt","feat_2a_250Hz_30_2/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A09T_30.txt","feat_2a_250Hz_30_2/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_3 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A03T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A03T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A03T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A03T_20.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A02T_30.txt","feat_2a_250Hz_30_2/feat_right_A02T_30.txt","feat_2a_250Hz_30_2/feat_left_A01T_30.txt","feat_2a_250Hz_30_2/feat_right_A01T_30.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A09T_30.txt","feat_2a_250Hz_30_2/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_4 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A07T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A07T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A07T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A07T_20.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A03T_30.txt","feat_2a_250Hz_30_2/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A01T_30.txt","feat_2a_250Hz_30_2/feat_right_A01T_30.txt","feat_2a_250Hz_30_2/feat_left_A09T_30.txt","feat_2a_250Hz_30_2/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_5 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A08T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A08T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A08T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A08T_20.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A03T_30.txt","feat_2a_250Hz_30_2/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_2/feat_left_A01T_30.txt","feat_2a_250Hz_30_2/feat_right_A01T_30.txt","feat_2a_250Hz_30_2/feat_left_A09T_30.txt","feat_2a_250Hz_30_2/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)

probability_2 = probability.reshape([5, 3])
weight_2t = (weight_1[:,0]+weight_2[:,0]+weight_3[:,0]+weight_4[:,0]+weight_5[:,0])/5

1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10


In [135]:
#train=20,test=40で分類を行った結果
probability = np.array([])
temp,temp1,temp2,weight_1 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A01T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A01T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A01T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A01T_20.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A02T_30.txt","feat_2a_250Hz_30_3/feat_right_A02T_30.txt","feat_2a_250Hz_30_3/feat_left_A03T_30.txt","feat_2a_250Hz_30_3/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A09T_30.txt","feat_2a_250Hz_30_3/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_2 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A02T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A02T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A02T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A02T_20.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A01T_30.txt","feat_2a_250Hz_30_3/feat_right_A01T_30.txt","feat_2a_250Hz_30_3/feat_left_A03T_30.txt","feat_2a_250Hz_30_3/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A09T_30.txt","feat_2a_250Hz_30_3/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_3 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A03T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A03T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A03T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A03T_20.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A02T_30.txt","feat_2a_250Hz_30_3/feat_right_A02T_30.txt","feat_2a_250Hz_30_3/feat_left_A01T_30.txt","feat_2a_250Hz_30_3/feat_right_A01T_30.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A09T_30.txt","feat_2a_250Hz_30_3/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_4 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A07T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A07T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A07T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A07T_20.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A03T_30.txt","feat_2a_250Hz_30_3/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A01T_30.txt","feat_2a_250Hz_30_3/feat_right_A01T_30.txt","feat_2a_250Hz_30_3/feat_left_A09T_30.txt","feat_2a_250Hz_30_3/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)
temp,temp1,temp2,weight_5 = SGRM_test("feat_2a_250Hz_20_test/feat_left_A08T_20_K0.txt","feat_2a_250Hz_20_test/feat_right_A08T_20_K0.txt","feat_2a_250Hz_20_test/feat_test_left_A08T_20.txt","feat_2a_250Hz_20_test/feat_test_right_A08T_20.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A03T_30.txt","feat_2a_250Hz_30_3/feat_right_A03T_30.txt"
                           ,"feat_2a_250Hz_30_3/feat_left_A01T_30.txt","feat_2a_250Hz_30_3/feat_right_A01T_30.txt","feat_2a_250Hz_30_3/feat_left_A09T_30.txt","feat_2a_250Hz_30_3/feat_right_A09T_30.txt")
temp_l = np.array([temp,temp1,temp2])
probability = np.append(probability,temp_l,axis=0)

probability_3 = probability.reshape([5, 3])
weight_3t = (weight_1[:,0]+weight_2[:,0]+weight_3[:,0]+weight_4[:,0]+weight_5[:,0])/5


1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10
1
2
3
4
5
6
7
8
9
10


In [144]:
p_average_250Hz = (probability_1+probability_2+probability_3)/3
weight_result = (weight_1t+weight_2t+weight_3t)/3
max_index = np.argmax(weight_result)
print(weight_result[max_index])
print(weight_1[max_index,1:3])
weight_result = weight_result.reshape([10,10])
pd.options.display.float_format = '{:.1f}'.format
feature1 = ["0.01","0.02","0.03","0.04","0.05","0.06","0.07","0.08","0.09","0.1"]
name1 = ["0.01","0.02","0.03","0.04","0.05","0.06","0.07","0.08","0.09","0.1"]
df = pd.DataFrame(weight_result,feature1,name1)
df

55.333333333333336
[0.01 0.06]


,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1
0.01,53.7,54.2,54.2,53.9,54.1,55.3,54.9,53.6,53.9,52.5
0.02,52.1,53.6,53.8,53.8,53.8,54.2,53.2,53.8,54.8,54.2
0.03,52.4,52.8,54.0,54.2,53.8,53.5,53.3,53.7,54.2,53.8
0.04,52.2,52.1,53.1,54.0,53.9,53.5,53.6,53.7,53.7,53.6
0.05,52.1,52.7,52.8,53.8,54.2,54.0,54.0,53.6,53.8,53.9
0.06,52.2,52.6,52.4,53.1,54.1,54.2,54.2,54.0,54.0,53.8
0.07,52.2,52.2,52.9,52.5,53.7,54.0,53.7,53.9,53.5,53.6
0.08,53.2,51.9,52.5,52.2,53.4,53.5,53.9,53.8,53.4,53.5
0.09,53.0,51.8,52.5,52.7,52.6,53.3,53.5,53.7,53.8,53.6
0.1,52.6,51.9,52.6,52.5,52.5,53.1,53.6,53.5,53.5,53.7
